# 1. Librerías & Set Up

👉🏻 Instalaremos todas las librerias necesarias para poder trabajar con PySpark y Neo4j

In [76]:
!pip install pyspark

In [77]:
!pip install neo4j

In [78]:
from pyspark.sql import SparkSession
from itertools import product

In [79]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [80]:
# Iniciamos una sesion en PySpark
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Neo4j Graphs and others examples

👉🏻 Las siguientes funciones permiten cargar grafos de ejemplos para poder realizar las consultas de los patrones de grafo.

👉🏻 Existe la funcion create_neo4j_graph para poder cargar una base de datos en Neo4j y luego obtener las aristas existentes de este grafo.

👉🏻 Existe la funcion load_graph para poder seleccionar el grafo que queremos realizar las consultas.

In [123]:
def load_neo4j_graph():
  """
  Funcion para cargar la base de datos de twitter como ejemplo de grafo en Neo4j.
  """
  # Realizamos la conexion con la base de datos del grafo guardado en Neo4j.
  NEO4J_URI = "neo4j+s://demo.neo4jlabs.com"
  NEO4J_USERNAME = "twitter"
  NEO4J_PASSWORD = "twitter"
  AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

  driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
  with driver.session() as session:
      try:
          session.run("RETURN 1")
          print("Connection to Neo4j established successfully!")
      except Exception as e:
          print(f"Failed to connect to Neo4j: {e}")
      # Query que permite cargar todas las aristas de cada usuario
      # (descartamos el usuario Neo4j para tener una cantidad mas variada xd,
      # ya que si lo dejabamos se repetia mucho)
      query = """
      MATCH (u1:User)-[r:INTERACTS_WITH | SIMILAR_TO]->(u2:User)
      WHERE u1.name <> "Neo4j" AND u2.name <> "Neo4j"
      RETURN u1.name as user1, type(r) AS relationship, u2.name AS user2
      """
      result = session.run(query)
      df_edges = pd.DataFrame(result.data())
      relationship = df_edges['relationship'].unique()

      # Definimos una clase para cada relacion ...
      relationship_class = {i : rel for rel, i in enumerate(relationship)}
      for inter, inter_class in relationship_class.items():
          print(f"{inter} AS class {inter_class}")
      df_edges['relationship'].replace(relationship_class, inplace = True)

      # Transformamos cada usuario a un numero (su identificador) en base
      # a su nombre de usuari que debe ser unico ...
      users1 = df_edges['user1'].unique().tolist()
      users2 = df_edges['user2'].unique().tolist()
      users = list(set(users1 + users2))
      users_id = {user : id for id, user in enumerate(users)}
      df_edges['user1'].replace(users_id, inplace = True)
      df_edges['user2'].replace(users_id, inplace = True)

      np.save('users_id', users_id, allow_pickle = True)
      np.save('relationship_class', relationship_class, allow_pickle = True)

      # Cargamos el grafo en una lista de tuplas
      neo4f_graph = [(item[0], item[1], item[2]) for item in df_edges.values]
  driver.close()
  return neo4f_graph


In [124]:
def load_graph(option):
  """
  Funcion que selecciona el grafo de ejemplo, retorna una lista de tuplas
  de las aristas del grafo.
  """
  if option == "neo4j-graph":
    neo4f_graph = load_neo4j_graph()
    return neo4f_graph

  elif option == "basic-example":
    return [
        (1,11,2), (1,11,3), (2,11,3), (3,11,2),
        (3,11,4), (4,11,1), (4,11,2), (4,11,3),
        (4,12,5), (5,12,1), (5,12,2), (5,12,6),
        (1,12,2), (2,12,3), (3,12,1)
    ]
  else:
    print("La opcion no existe :(")
    return None

# 3. MapReduce Algorithm Implementation

👉🏻 Las siguientes funciones permiten desarrollar el algoritmo MapReduce implementado con PySpark

In [125]:
def hash(n):
  """
  Returns number mod 2. The ouput will be 0 or 1.
  """
  return n % 2

👉🏻 Las funciones siguientes permiten simular la fase de mapeo del algoritmo.

In [126]:
def get_keys(edge, b_dim, b_set, pattern_dim):
  """
  output: retorna las llaves correspondientes para un vertice.

  Idea general: buscamos el par hash_n1,hash_n2 dentro de las posibles
  combinaciones dentro del espacio de imagenes de la funcion de hash.
  Dentro del for, obtenemos un string con la codificacion de las llaves y luego
  verificamos si es una llave candidata para el vertice entregado:

  1.  sequence_in_reducer: Si la secuencia 'b1b2' esta en la llave del reducer
      codificada como hash(n1)hash(n2) ?, donde ? = 0 o 1, entonces se considerará
      el par reducer_key : edge.
  2.  edge_case: El otro caso, es para cuando tenemos por ejemplo x = n2 y z = n1
      para patrones de 3 vertices.
  """

  hash_n1 = hash(edge[0]) # valor de hash para el nodo 1 = b1
  hash_n2 = hash(edge[2]) # valor de hash para el nodo 2 = b2
  values = [] # posible keys
  sequence = '{}{}'.format(hash_n1, hash_n2)

  for i in range(0, b_dim ** pattern_dim):
    reducer = ''.join(str(num) for num in b_set[i])
    sequence_in_reducer = sequence in reducer
    edge_case = reducer[0] == sequence[1] and reducer[pattern_dim - 1] == sequence[0]
    if sequence_in_reducer or edge_case:
        reducer_key = tuple(int(digit) for digit in reducer)
        values.append((reducer_key, edge))

  return values


def map_phase(rdd, b_dim, b_set, pattern_dim):
  """
  input:
    - rdd: RDD del grafo de dimension 'dim'
    - b_dim: Cantidad de elementos de las imagenes de la funcion de hash.
    - b_set: Imagenes de la funcion de hash.
    - pattern_dim: cantidad de nodos del patron de grafo.
  ouput: Mapeo de cada arista con respecto a las llaves
  """

  mapped_keys = rdd.flatMap(lambda edge: get_keys(edge, b_dim, b_set, pattern_dim))

  return mapped_keys

👉🏻 Las siguientes funciones permiten simular la fase de reduce del algoritmo.

In [127]:
def find_patterns(edges, pattern_dim):
    """
    Funcion que retorna todos los patrones encontrados de la forma
    (n1,...,nl) : [[(n1,label,n2), ..., (nl-1,label,nl)], ... ]
    donde (n1,...,nl) corresponde a los nodos que forman el patron
    y la llave corresponde a los patrones posibles formados por estos nodos
    para una etique
    """
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim)
    return cycles


def get_neighbors(edges):
    """
    Funcion que retorna todas las aristas consecutivas a otras aristas.
    Es decir, aristas vecinas.
    """
    neighbors = []
    for i in range(len(edges)):
        curr_edge = edges[i]
        neighbors.append([])
        for j in range(len(edges)):
            next_edge = edges[j]
            if next_edge == curr_edge:
                continue
            if curr_edge[2] == next_edge[0]:
                neighbors[i].append(next_edge)
    return neighbors


def find_cycles(edges, neighbors, pattern_dim):
    """
    Funcion que verifica si existe algun grafo ciclico en el grafo de una
    dimension pattern_dim. Retorna todos los ciclos encontrados.
    Utilizamos el algoritmo DFS iterativo para la busqueda de ciclos.
    """
    cycles = [] # Lista de ciclos encontrados

    for edge in edges: # Por cada arista del grafo
        visited = set() # Determinamos las aritas que ya han sido visitadas
        # Inicializamos el stack con con la
        stack = [(edge, [edge])]
        # arista donde empezamos el recorrido

        while len(stack):
            curr_edge, path = stack.pop() # Extraemos la primera arista del stack
            # Recorremos esta arista, por lo que la marcamos como visitada
            visited.add(curr_edge)
            # Caso 1: Si el camino recorrido actual supera la cantidad de nodos del bgp,
            #         descartamos este camino
            if len(path) > pattern_dim:
                continue
            # Caso 2: Si el camino recorrido tiene un largo igual a pattern_dim,
            #         y son aristas transitivas, entonces un posible ciclo.
            first_node = path[0][0] # primer nodo del camino
            last_node = path[-1][2] # ultimo nodo del caminmo
            if len(path) == pattern_dim and first_node == last_node:
                cycles.append(path)
                # Agregamos el camino y seguimos recorriendo
                continue
            # Si no es ciclico el camino desde el nodo actual,
            # entonces iteramos dentro de sus vecinos
            for neighbor_edge in neighbors[edges.index(curr_edge)]:
                if neighbor_edge not in visited:
                    stack.append((neighbor_edge, path + [neighbor_edge]))

    return cycles


def get_nodes(pattern, pattern_dim):
    """
    Funcion que retorna la tupla con los nodos que forman un ciclo.
    """
    nodes = []
    for i in range(pattern_dim):
      edge = pattern[i]
      nodes.append(edge[0])

    return (tuple(sorted(nodes)), tuple(sorted(pattern)))


def get_unique_lists(pattern1, pattern2):
    """
    Funcion que elimina patrones de grafo duplicados para cada llave
    (n1,n2, ..., nl). Si tenemos l = 3, entonces seria (n1,n2,n3)
    """
    lst = []
    if pattern2 != pattern1:
      return lst.append(pattern2)
    else:
      return pattern1


def reduce_phase(mapped_keys, pattern_dim):
    """
    input: RDD del grafo y cantidad de nodos del patron de grafo.
    output: patrones encontrados.
    """
    # Agrupamos por llaves
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Filtramos solamente los grupos conformados por 3 aristas ...
    reducers_edges = reducers.map(lambda v: v[1]) \
                    .filter(lambda x: len(x) >= pattern_dim)
    # Por cada grupo buscamos un patron de grafo
    patterns = reducers_edges \
              .map(lambda edges: find_patterns(edges, pattern_dim)) \
              .flatMap(list) \
              .map(lambda pattern: get_nodes(pattern, pattern_dim)) \
              .groupByKey().mapValues(lambda v: list(set(v)))


    return patterns

👉🏻 Funcion final que simula el algoritmo MapReduce. Recibe los parametros correspondientes del grafo; las dimensiones del conjunto de imagenes de la funcion de hash; el conjunto de imagenes de la funcion de hash; la dimension del patron de grafo.

In [128]:
def map_reduce(rdd_graph, b_dim, b_set, pattern_dim, show=False):
  """
  Funcion que simula el algoritmo MapReduce, en donde se distribuye el grafo
  en diferentes reducers y luego obtenemos los posibles patrones
  de grafos formados combinando las informacion de todos los reducers.
  """
  # Fase de Map: Obtenemos las llaves de cada reducer y el conjunto de aristas mapeados a estas llaves.
  mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)

  if show: # Mostras las llaves mapeadas
    print("Fase Map: \n")
    for key in mapped_keys.collect():
      print(key)
    print("\nFase Reduce\n")

  # Fase Reduce: Obtenemos todos los posibles patrones de L nodos.
  patterns = reduce_phase(mapped_keys, pattern_dim)
  return patterns

# 3. Aplicacion MapReduce en Ejemplo Basico

👉🏻 En esta seccion aplicaremos el algoritmo MapReduce para un grafo de prueba pequeño (el que fue dado al inicio de la tarea, con algunas modificaciones). Este lo guardamos como 'basic-example'.

In [129]:
graph = load_graph('basic-example')
rdd_graph = sc.parallelize(graph)
rdd_graph.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6),
 (1, 12, 2),
 (2, 12, 3),
 (3, 12, 1)]

## 3.1 MapReduce Algorithm for Triangles

👉🏻 Para encontrar triangulos dentro de un grafo utilizaremos el algoritmo Map Reduce para parametros b_0 = 2 y l = 3.

👉🏻 En este caso, b_0=2 ya que al utilizar la funcion de hash modular `f(x) = x mod 2`, generamos un conjunto de 2 imagenes: {0,1}. En consecuencia, los posibles reducers son:
```markdown
REDUCERS = [(0, 0, 0),
            (0, 0, 1),
            (0, 1, 0),
            (0, 1, 1),
            (1, 0, 0),
            (1, 0, 1),
            (1, 1, 0),
            (1, 1, 1)]
```

👉🏻 Dado que estamos buscando subgrafos de 3 nodos, entonces l_0 = 3.

👉🏻 Las aristas se distribuyen dentro de los reducers, por lo que almacenamos el grafo de manera distribuida.En cada reducer verificamos si las aristas generan triangulos, en el caso que si (formaria un ciclo) se almacenará en una lista y retornaremos los nodos correspondientes.




In [130]:
# Dimension de elementos del conjunto de imagenes de la funcion de hash: |{0,1}|
b_0 = 2
# Dimension del patron de grafo (triangulo para este caso)
l_0 = 3
# Conjunto de imagenes de la funcion de hash
b_set_0 = list(range(b_0))
# Posibles reducers
reducers_0 = list(product(b_set_0, repeat=l_0))

patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)

patterns.collect()

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

👉🏻 Al aplicar el algoritmo mapReduce, podemos notar que existen 3 tripletas de nodos que forman un triangulo. En donde estos nodos permiten formar diferentes patrones de grafos triangulares. En este caso, como tenemos mas de una etiqueta distinta, podemos notar que para una tripleta de nodo, forman mas de un triangulo.
```markdown
  (2,3,4); (1,2,3); (1,3,4)
```

👉🏻 Las siguientes funciones se definiran con el fin de desarrollar consultas sobre estos grafos.


In [131]:
def filter_patterns(pattern, pattern_dim, labels):
  """
  Funcion que retorna los patrones filtrados segun el label proporcionado como
  parametro.
  """
  pattern_labels = [edge[1] for edge in pattern]
  for i in range(pattern_dim + 1):
      # verificamos si coinciden los labels ...
      if pattern_labels == labels:
        return pattern
      # Hacemos un desplazamiento hacia la derecha ...
      pattern_labels = pattern_labels[i:] + pattern_labels[:i]

  return None

def bgp_query(rdd_graph, b_dim, reducers, pattern_dim, query):
  """
  Funcion que retorna todos los patrones de grafos de dimension "pattern_dim"
  que coincidan segun la query entregada.
  """
  patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)
  processed_query = query.replace("(", "").replace(")", "").split(",")
  labels = [int(part.strip()) for part in processed_query if part.strip().isdigit()]

  result = patterns.filter(lambda pattern: filter_patterns(pattern, pattern_dim, labels))
  return result

In [132]:
# Realizamos la siguiente consulta en donde queremos obtener todos los triangulos
# donde la etiqueta es 11, independiente del nodo.
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,11,y), (y,11,z), (z,11,w)")
for res in result.collect():
  print(res)

((2, 11, 3), (3, 11, 4), (4, 11, 2))
((1, 11, 3), (3, 11, 4), (4, 11, 1))


## 3.2 MapReduce Algorithm for Squares


👉🏻 Ahora aplicaremos nuestro algoritmo para la busqueda de patrones de grafos de 4 variables. En primer lugar, utilizaremos el mismo algoritmo anterior, pero cambiaremos los parametros de l y b_set. En este caso, utilizaremos l = 4. En consecuencia, obtenemos una cantidad de $2^4$ reducers (definidos como `reducers_1`).

In [133]:
# Como buscaremos los posibles cuadrados a formar, entonces tendremos l=4 nodos
# en el bgp.
l_1 = 4
# Tendremos b^4 reducers posibles.
b_set_1 = list(range(b_0))
reducers_1 = list(product(b_set_1, repeat=l_1))
# aplicamos el algoritmo para patrones de grafos de l=4 nodos, con 2^4 reducers
# y con un conjunto de preimagenes de la funcion de hash {0,1}.
squares_patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
squares_patterns.collect() # Obtenemos todos los posibles patrones cuadrados sin importar el label.

[((1, 2, 3, 4),
  [((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1))]),
 ((2, 3, 4, 5),
  [((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2))]),
 ((2, 3, 3, 4),
  [((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3)),
   ((2, 12, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))]),
 ((1, 3, 4, 5), [((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))]),
 ((1, 2, 3, 3), [((1, 11, 3), (2, 12, 3), (3, 11, 2), (3, 12, 1))])]

👉🏻 Ahora podemos notar que el algoritmo se demora aproximadamente 1s en correr.

👉🏻 Podemos notar que existen 5 tuplas que forman patrones de grafos cuadrilateros (considerando que tenemos diferentes etiquetas, por lo que algunos nodos se repiten, sin embargo se debe a que presentan labels de aristas distintos).

```markdown
(1,2,3,4), (2,3,4,5), (2,3,3,4), (1,2,4,5), y (1,2,3,3).
```

In [134]:
result = bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,11,w), (w,11,x)")
for res in result.collect():
  print(res)

((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1))
((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))


In [135]:
result = bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,12,w), (w,12,x)")
for res in result.collect():
  print(res)

((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1))
((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2))
((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))


# 4. Aplicacion de un grafo a gran escala: Twitter Graph

👉🏻 Ahora cargaremos un grafo de mayor tamaño (700 aristas) para evaluar nuestro algoritmo. El siguiente grafo corresponde a un grafo que modela la interaccion de cada usuario en la red social twitter.
El grafo sera cargado como lista de tuplas, en donde cada tupla modela la relacion entre un usuario y otro. Consideramos las siguientes 2 tipos de relaciones

    Relacion 0: INTERACTS_WITH - El usuario 1 interacciona con otro usuario 2
    Relacion 1: SIMILAR_TO - El usuario 1 es similar al usuario 2

Referencia: https://github.com/neo4j-graph-examples/twitter-v2


In [136]:
graph = load_graph('neo4j-graph')
rdd_graph = sc.parallelize(graph)

Connection to Neo4j established successfully!
INTERACTS_WITH AS class 0
SIMILAR_TO AS class 1


In [137]:
rdd_graph.count() # Cantidad de aristas del grafo

744

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Ahora aplicaremos MapReduce para encontrar los patrones de grafos con forma de triangulo dentro del grafo cargado desde Neo4j.



In [138]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)
patterns.collect()

[((297, 467, 487), [((297, 1, 487), (467, 1, 297), (487, 0, 467))]),
 ((47, 329, 487), [((47, 1, 487), (329, 1, 47), (487, 0, 329))]),
 ((47, 297, 467), [((47, 0, 467), (297, 1, 47), (467, 1, 297))]),
 ((47, 401, 487), [((47, 1, 487), (401, 1, 47), (487, 0, 401))]),
 ((47, 467, 487), [((47, 1, 487), (467, 1, 47), (487, 0, 467))]),
 ((143, 345, 451),
  [((143, 0, 451), (345, 0, 143), (451, 1, 345)),
   ((143, 1, 345), (345, 0, 451), (451, 1, 143))]),
 ((181, 239, 263),
  [((181, 1, 239), (239, 0, 263), (263, 1, 181)),
   ((181, 0, 263), (239, 0, 181), (263, 1, 239))]),
 ((151, 187, 317), [((151, 0, 187), (187, 1, 317), (317, 1, 151))]),
 ((69, 371, 449), [((69, 0, 449), (371, 1, 69), (449, 1, 371))]),
 ((3, 69, 483), [((3, 0, 483), (69, 0, 3), (483, 1, 69))]),
 ((3, 119, 371), [((3, 0, 119), (119, 0, 371), (371, 1, 3))]),
 ((3, 371, 449), [((3, 0, 449), (371, 1, 3), (449, 1, 371))]),
 ((3, 119, 483), [((3, 0, 483), (119, 1, 3), (483, 1, 119))]),
 ((3, 69, 119), [((3, 0, 119), (69, 0, 3)

👉🏻 El algoritmo es bien rapido de ejecutar, y podemos ver claramente las relaciones entre diferentes usuarios, para diferentes labels.

👉🏻 Para poder analizar las relaciones y compararlo con el grafo mostrado en Neo4j, realizaremos una consulta, y luego compararemos los resultados ...

In [139]:
# Cargamos el diccionario que guardamos los nombres de usuarios y el id asociado.
users_id = np.load('users_id.npy', allow_pickle=True).item()
dict_users_id = {value : key for key, value in users_id.items()}
# Cargamos el tipo de relacion.
relationship_class = np.load('relationship_class.npy', allow_pickle=True).item()
dict_relation_class = {value : key for key, value in relationship_class.items()}

In [140]:
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,1,y), (y,0,z), (z,0,w)")
result.collect()

[((3, 0, 483), (371, 1, 3), (483, 0, 371)),
 ((47, 0, 302), (160, 1, 47), (302, 0, 160)),
 ((69, 0, 504), (308, 1, 69), (504, 0, 308)),
 ((69, 0, 308), (308, 1, 318), (318, 0, 69)),
 ((28, 1, 487), (401, 0, 28), (487, 0, 401)),
 ((151, 0, 187), (187, 1, 298), (298, 0, 151)),
 ((69, 0, 324), (225, 1, 69), (324, 0, 225)),
 ((3, 1, 69), (69, 0, 308), (308, 0, 3)),
 ((294, 0, 483), (371, 1, 294), (483, 0, 371)),
 ((294, 0, 483), (449, 1, 294), (483, 0, 449))]

In [141]:
def load_usernames(pat):
  """
  Funcion que retorna el nombre de usuario codificado a un id, y tambien transforma
  la clase del label a su valor original
  """
  list_users = []
  for i in range(len(pat)):
    edge = pat[i]
    user1 = dict_users_id[edge[0]]
    label = dict_relation_class[edge[1]]
    user2 = dict_users_id[edge[2]]
    list_users.append((user1, label, user2))
  return list_users

👉🏻 Ahora con la consulta anterior, analizaremos los usuarios involucrados en el siguiente caso: el primer usuario interacciona con el segundo; el segundo es similar al tercero; y el tercero el similar al primero.

In [142]:
result.map(load_usernames).collect()

[[('John Wolpert', 'INTERACTS_WITH', 'Brana Rakic'),
  ('Wen.S', 'SIMILAR_TO', 'John Wolpert'),
  ('Brana Rakic', 'INTERACTS_WITH', 'Wen.S')],
 [('Michael Hunger', 'INTERACTS_WITH', 'Sandro Miccoli'),
  ('CRYPTEMES', 'SIMILAR_TO', 'Michael Hunger'),
  ('Sandro Miccoli', 'INTERACTS_WITH', 'CRYPTEMES')],
 [('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs',
   'INTERACTS_WITH',
   'Chidambara .ML.'),
  ('Nancy Tremblay❤😀🌻🇨🇦',
   'SIMILAR_TO',
   'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs'),
  ('Chidambara .ML.', 'INTERACTS_WITH', 'Nancy Tremblay❤😀🌻🇨🇦')],
 [('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs',
   'INTERACTS_WITH',
   'Nancy Tremblay❤😀🌻🇨🇦'),
  ('Nancy Tremblay❤😀🌻🇨🇦', 'SIMILAR_TO', 'Lju Lazarevic'),
  ('Lju Lazarevic',
   'INTERACTS_WITH',
   'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs')],
 [('Nigel Small 🇪🇺', 'SIMILAR_TO', 'Michael Simons'),
  ('Jim Webber', 'INTERACTS_WITH', 'Nigel Small 🇪🇺'),
  ('Michael Simons', 'INTERACTS_WITH', 'Jim Webber')],
 [('Sonja Heward-mills', 'INTERACTS_WITH', 

## 4.2 MapReduce For Squares

👉🏻 Ahora aplicaremos el algoritmo MapReduce para buscar patrones todos los posibles patrones de grafos de triangulos, para luego realizar una consulta.

In [143]:
patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
patterns.collect()

[((28, 218, 487, 487),
  [((28, 1, 487), (218, 1, 487), (487, 0, 28), (487, 0, 218))]),
 ((297, 467, 467, 487),
  [((297, 0, 467), (467, 1, 297), (467, 1, 487), (487, 0, 467))]),
 ((47, 329, 329, 487),
  [((47, 0, 329), (329, 1, 47), (329, 1, 487), (487, 0, 329))]),
 ((329, 401, 487, 487),
  [((329, 1, 487), (401, 1, 487), (487, 0, 329), (487, 0, 401))]),
 ((401, 467, 487, 487),
  [((401, 1, 487), (467, 1, 487), (487, 0, 401), (487, 0, 467))]),
 ((289, 318, 318, 471),
  [((289, 1, 318), (318, 0, 289), (318, 0, 471), (471, 1, 318))]),
 ((69, 318, 318, 471),
  [((69, 1, 318), (318, 0, 69), (318, 0, 471), (471, 1, 318))]),
 ((69, 289, 318, 318),
  [((69, 1, 318), (289, 1, 318), (318, 0, 69), (318, 0, 289))]),
 ((308, 318, 318, 374),
  [((308, 1, 318), (318, 0, 308), (318, 0, 374), (374, 1, 318))]),
 ((47, 318, 318, 471),
  [((47, 1, 318), (318, 0, 47), (318, 0, 471), (471, 1, 318))]),
 ((47, 289, 318, 318),
  [((47, 1, 318), (289, 1, 318), (318, 0, 47), (318, 0, 289))]),
 ((69, 225, 318, 

👉🏻 Realizamos la siguiente consulta en donde obtener todos los usuarios tal que:
* El primero interacciona con el segundo
* El segundo interaccion con el tercero
* El tercero es similar al cuarto
* El cuarto es similar al primero

In [144]:
result = bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,0,z), (z,1,w), (w,1,x)")
for res in result.collect():
  print(res)

((28, 1, 487), (218, 1, 487), (487, 0, 28), (487, 0, 218))
((297, 0, 467), (467, 1, 297), (467, 1, 487), (487, 0, 467))
((47, 0, 329), (329, 1, 47), (329, 1, 487), (487, 0, 329))
((329, 1, 487), (401, 1, 487), (487, 0, 329), (487, 0, 401))
((401, 1, 487), (467, 1, 487), (487, 0, 401), (487, 0, 467))
((289, 1, 318), (318, 0, 289), (318, 0, 471), (471, 1, 318))
((69, 1, 318), (318, 0, 69), (318, 0, 471), (471, 1, 318))
((69, 1, 318), (289, 1, 318), (318, 0, 69), (318, 0, 289))
((308, 1, 318), (318, 0, 308), (318, 0, 374), (374, 1, 318))
((47, 1, 318), (318, 0, 47), (318, 0, 471), (471, 1, 318))
((47, 1, 318), (289, 1, 318), (318, 0, 47), (318, 0, 289))
((69, 1, 318), (225, 1, 69), (318, 0, 324), (324, 0, 225))
((47, 0, 302), (47, 0, 362), (302, 1, 47), (362, 1, 47))
((47, 0, 82), (47, 0, 362), (82, 1, 47), (362, 1, 47))
((47, 0, 329), (47, 0, 467), (329, 1, 47), (467, 1, 47))
((47, 0, 309), (47, 0, 329), (309, 1, 47), (329, 1, 47))
((47, 0, 309), (47, 0, 467), (309, 1, 47), (467, 1, 47))

👉🏻 Ahora realizamos otra consulta en donde el primer interacciona con el segundo; el segundo es similar al tercero; el tercero es similar al cuarto; el cuarto es similar al primero.

In [145]:
result = bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,1,z), (z,1,w), (w,1,x)")
for res in result.collect():
  print(res)

((3, 1, 69), (69, 0, 449), (371, 1, 3), (449, 1, 371))
((69, 1, 318), (308, 1, 327), (318, 0, 308), (327, 1, 69))
((294, 1, 308), (308, 0, 449), (371, 1, 294), (449, 1, 371))
((47, 1, 487), (297, 1, 47), (467, 1, 297), (487, 0, 467))
((47, 1, 318), (318, 1, 487), (329, 1, 47), (487, 0, 329))
((47, 1, 318), (69, 1, 333), (318, 0, 69), (333, 1, 47))
((47, 1, 318), (318, 1, 487), (401, 1, 47), (487, 0, 401))
((47, 1, 318), (318, 1, 487), (467, 1, 47), (487, 0, 467))
((69, 1, 318), (308, 1, 504), (318, 0, 308), (504, 1, 69))
((3, 1, 308), (308, 0, 449), (371, 1, 3), (449, 1, 371))
((3, 1, 294), (294, 0, 449), (371, 1, 3), (449, 1, 371))
((119, 1, 308), (308, 0, 449), (371, 1, 119), (449, 1, 371))
((3, 1, 294), (119, 1, 3), (294, 0, 483), (483, 1, 119))
((308, 0, 449), (371, 1, 483), (449, 1, 371), (483, 1, 308))


👉🏻 Ahora, para poder analizar quienes son los usuarios involucrados en la ultima consulta, mapearemos cada id con respecto a su username :)

In [146]:
result.map(load_usernames).collect()

[[('John Wolpert', 'SIMILAR_TO', 'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs'),
  ('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs',
   'INTERACTS_WITH',
   'guinnessstache'),
  ('Wen.S', 'SIMILAR_TO', 'John Wolpert'),
  ('guinnessstache', 'SIMILAR_TO', 'Wen.S')],
 [('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs', 'SIMILAR_TO', 'Lju Lazarevic'),
  ('Nancy Tremblay❤😀🌻🇨🇦', 'SIMILAR_TO', 'Crypto Fido'),
  ('Lju Lazarevic', 'INTERACTS_WITH', 'Nancy Tremblay❤😀🌻🇨🇦'),
  ('Crypto Fido', 'SIMILAR_TO', 'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs')],
 [('Born To Be Idle', 'SIMILAR_TO', 'Nancy Tremblay❤😀🌻🇨🇦'),
  ('Nancy Tremblay❤😀🌻🇨🇦', 'INTERACTS_WITH', 'guinnessstache'),
  ('Wen.S', 'SIMILAR_TO', 'Born To Be Idle'),
  ('guinnessstache', 'SIMILAR_TO', 'Wen.S')],
 [('Michael Hunger', 'SIMILAR_TO', 'Michael Simons'),
  ('Stefan Keller', 'SIMILAR_TO', 'Michael Hunger'),
  ('Lukas Eder', 'SIMILAR_TO', 'Stefan Keller'),
  ('Michael Simons', 'INTERACTS_WITH', 'Lukas Eder')],
 [('Michael Hunger', 'SIMILAR_TO', 'Lju La